In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

/home/caihengxing/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
path = '/home/caihengxing/taac_data/bank/train_data/'

In [3]:
os.listdir(path)

['overdue_train.csv',
 'bill_train.csv',
 'browse_train.csv',
 'userinfo_train.csv',
 'bank_train.csv']

## Main Table

In [4]:
userinfo = pd.read_csv(f"{path}/userinfo_train.csv")
userinfo.columns = ['用户id', '性别', '职业', '教育程度', '婚姻状态', '户口类型']

In [5]:
overdue = pd.read_csv(f"{path}/overdue_train.csv")
del_cols = ['flag1']
overdue.drop(del_cols, axis=1, inplace=True)
overdue.columns = ['用户id', '标签']

In [6]:
df = pd.merge(userinfo, overdue, on = '用户id', how = 'left')

In [7]:
df.head()

,用户id,性别,职业,教育程度,婚姻状态,户口类型,标签
0,eddcaa8984f8db5199ec28323efb18a4,1,2,4,1,4,0
1,966685f4d34386f3ea23cbb72d01cd94,1,2,4,3,2,0
2,65b01fba105ee82613babff7c88929c5,1,3,4,3,1,0
3,d721497ae58b1ab5164d1b39263248c0,1,2,4,3,2,0
4,e69cf2ce4683229904f05fc4c8c7a003,2,2,2,1,1,0


In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [9]:
train.shape, test.shape

((28664, 7), (7167, 7))

In [10]:
# train = train.sample(frac = 0.1, random_state = 42)
# test  = test.sample(frac = 0.1, random_state = 42)

In [11]:
train.index = range(len(train))
test.index = range(len(test))

In [12]:
test.drop('标签', axis=1, inplace=True)

/home/caihengxing/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
train.head()

,用户id,性别,职业,教育程度,婚姻状态,户口类型,标签
0,f34cf3be330734cac1cdc5503d06be5b,1,4,4,1,1,0
1,a6ef3c341d386d3d75aefa9500355625,1,2,4,1,4,0
2,6ee809a636411b5322646e98157ed017,1,2,4,1,4,0
3,dc68a34468131dd4f9a27c75e6dd5026,1,2,4,1,1,0
4,824e90bd82f905216cc03ad62c8195b3,1,2,3,3,1,1


In [14]:
test.head()

,用户id,性别,职业,教育程度,婚姻状态,户口类型
0,42a16105b10ee12782ff39f1598ac70e,1,2,3,3,1
1,405b004aaa14b4a6017a2f44686e3440,2,2,3,2,1
2,5c17da3e7807825fa57716f289a36634,2,2,4,2,1
3,6f8d122b215074ca23b1980f85afd4c8,1,4,4,3,3
4,a49138b7665cc8093353bebca84f7701,1,2,3,2,2


## Secondary Table

In [15]:
bank = pd.read_csv(f"{path}/bank_train.csv")
bank.columns = ['用户id','时间戳','交易类型','交易金额','工资收入标记']

In [16]:
import random
randomlist = []
for i in range(0,len(bank)):
    n = random.randint(1,6)
    randomlist.append(n)

In [17]:
bank['月份'] = randomlist

In [18]:
del_list = ["时间戳"]
bank.drop(del_list, axis = 1, inplace = True)

In [19]:
bank.head()

,用户id,交易类型,交易金额,工资收入标记,月份
0,966685f4d34386f3ea23cbb72d01cd94,0,45.559482,0,6
1,966685f4d34386f3ea23cbb72d01cd94,1,45.559482,0,3
2,966685f4d34386f3ea23cbb72d01cd94,0,46.313940,0,6
3,966685f4d34386f3ea23cbb72d01cd94,1,41.866171,0,2
4,966685f4d34386f3ea23cbb72d01cd94,1,45.443705,0,4


In [20]:
bank.shape

(4479878, 5)

In [21]:
Id = '用户id'
bank_train = bank.loc[bank[Id].isin(train[Id])]
bank_test = bank.loc[bank[Id].isin(test[Id])]

In [22]:
bank_train.index = range(len(bank_train))
bank_test.index = range(len(bank_test))

In [23]:
bank_train.shape, bank_test.shape

((3556821, 5), (923057, 5))

In [24]:
train = train.loc[train[Id].isin(bank_train[Id].unique())]
test = test.loc[test[Id].isin(bank_test[Id].unique())]

train = train.sort_values(by = Id)
test = test.sort_values(by = Id)

bank_train = bank_train.sort_values(by = [Id, '月份'])
bank_test = bank_test.sort_values(by = [Id, '月份'])

train.index = range(len(train))
test.index = range(len(test))

bank_train.index = range(len(bank_train))
bank_test.index = range(len(bank_test))

In [25]:
target = '标签'

In [27]:
train.head()

,用户id,性别,职业,教育程度,婚姻状态,户口类型,标签
0,000014b8ec0ce8ad7c20f56915fc3a9f,1,2,3,1,2,0
1,0003f283dfacd7100bba76d876cf94da,1,4,4,1,2,0
2,0015f0bf7222ad1b2a96612d752552c3,1,2,4,2,2,0
3,00251f8fa9f3e014339d90e4dba1affd,1,2,4,3,1,0
4,00284cf15ae27d1ddf4f93922cd7bcb5,1,2,4,3,1,0


In [28]:
bank_train.head()

,用户id,交易类型,交易金额,工资收入标记,月份
0,000014b8ec0ce8ad7c20f56915fc3a9f,1,38.134741,0,1
1,000014b8ec0ce8ad7c20f56915fc3a9f,1,40.189051,0,1
2,000014b8ec0ce8ad7c20f56915fc3a9f,1,42.143743,0,1
3,000014b8ec0ce8ad7c20f56915fc3a9f,1,40.125193,0,1
4,000014b8ec0ce8ad7c20f56915fc3a9f,1,39.317371,0,1


In [29]:
train.shape, bank_train.shape

((5475, 7), (3556821, 5))

In [30]:
test.head()

,用户id,性别,职业,教育程度,婚姻状态,户口类型
0,0007be49a0f33d74e6ffbe3b6b0b8a96,1,2,3,1,3
1,00872649357929f1f3d7fafbe484810b,0,0,0,0,0
2,00bde750bb47544d97ba792e03d23ff7,1,2,4,1,1
3,00d4ecc55ecc3099e34c5bd60f5bfc6c,1,2,3,3,1
4,010424165f66c248d7ac1b3470850933,1,2,3,1,1


In [31]:
bank_test.head()

,用户id,交易类型,交易金额,工资收入标记,月份
0,0007be49a0f33d74e6ffbe3b6b0b8a96,0,50.865021,0,1
1,0007be49a0f33d74e6ffbe3b6b0b8a96,0,51.665283,0,1
2,0007be49a0f33d74e6ffbe3b6b0b8a96,1,43.388533,0,1
3,0007be49a0f33d74e6ffbe3b6b0b8a96,1,42.143743,0,1
4,0007be49a0f33d74e6ffbe3b6b0b8a96,1,44.499610,0,1


In [32]:
train.columns

Index(['用户id', '性别', '职业', '教育程度', '婚姻状态', '户口类型', '标签'], dtype='object')

In [33]:
train_a = train[['用户id', '性别', '职业', '标签']]
train_b = train[['用户id', '教育程度', '婚姻状态', '户口类型']]

In [34]:
test_a = test[['用户id', '性别', '职业']]
test_b = test[['用户id', '教育程度', '婚姻状态', '户口类型']]

In [35]:
# os.mkdir('./bank')

In [36]:
train_a.to_csv('./bank/train.csv', index = False)
test_a.to_csv('./bank/test.csv', index = False)

In [37]:
train_b.to_csv('./bank/train_info.csv', index = False)
test_b.to_csv('./bank/test_info.csv', index = False)

In [38]:
bank_train.to_csv('./bank/bank_train.csv', index = False)
bank_test.to_csv('./bank/bank_test.csv', index = False)